## Primeiros passos com Keras

Esta contribuição demonstra a facilidade do desenvolvimento de uma rede neural convolucional LeNet-5 utilizando framework keras com TensorFlow em background e utilizando o dataset Cifar10.
 

In [54]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
import warnings
#import matplotlib.pyplot as plt

#%matplotlib inline
warnings.simplefilter("ignore")

##### Carrega dataset Cifar10 e faz o split em treino e test.

In [19]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

170500096/170498071 [==============================] - 261s 2us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


##### Parâmetros da rede e de treinamento

In [38]:
batch_size = 32
num_classes = 10
epochs = 10
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'Keras_Cifar10_Model'
INPUT_SHAPE = x_train.shape[1:]

In [21]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

##### Criação da rede convolucional

In [68]:
model = Sequential()
model.add(Conv2D(32,(3,3), padding='same', input_shape=INPUT_SHAPE))
model.add(Activation('relu'))
model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

##### Adicionando função de otimização

In [69]:
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

##### Treinando a rede

In [70]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [71]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [72]:
if not data_augmentation:
    print('Not using data augmentation')
    model.fit(x_train, y_train,
             batch_size=batch_size,
             epochs=epochs,
             validation_data=(y_test,x_test),
             shuffle=True)
else:
    print('Using-real-time data augmentation')
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        zca_epsilon=1e-6,
        rotation_range=0,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.,
        zoom_range=0.,
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0) 

Using-real-time data augmentation


In [73]:
datagen.fit(x_train)

In [ ]:
model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    epochs=epochs,
                    validation_data=(x_test,y_test),
                    workers=4,
                    steps_per_epoch=10)

Epoch 1/10
10/10 [==============================] - 21s 2s/step - loss: 2.3027 - acc: 0.0750 - val_loss: 2.3026 - val_acc: 0.1000
Epoch 2/10
10/10 [==============================] - 18s 2s/step - loss: 2.3025 - acc: 0.1250 - val_loss: 2.3026 - val_acc: 0.1000
Epoch 3/10
10/10 [==============================] - 17s 2s/step - loss: 2.3027 - acc: 0.1156 - val_loss: 2.3026 - val_acc: 0.1000
Epoch 4/10
10/10 [==============================] - 19s 2s/step - loss: 2.3031 - acc: 0.0813 - val_loss: 2.3026 - val_acc: 0.1000
Epoch 5/10
10/10 [==============================] - 20s 2s/step - loss: 2.3024 - acc: 0.1125 - val_loss: 2.3026 - val_acc: 0.1000
Epoch 6/10
10/10 [==============================] - 23s 2s/step - loss: 2.3023 - acc: 0.1281 - val_loss: 2.3026 - val_acc: 0.1000
Epoch 7/10
 9/10 [==========================>...] - ETA: 0s - loss: 2.3026 - acc: 0.0938

##### Save Model

In [41]:
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

model_path = os.path.join(save_dir,model_name)
model.save(model_path)
print('Modelo treinado e salvo em %s ' % model_path)

Modelo treinado e salvo em C:\Users\leandro.r.a.silvA\MachineLearning-Leandro\CNN\saved_models\Keras_Cifar10_Model 


##### Perda e Acurácia do modelo aplicado aos dados de teste

In [42]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Perda em teste:', score[0])
print('Acurácia em teste:', score[1])

10000/10000 [==============================] - 12s 1ms/step
Perda em teste: 1.8834910352706908
Acurácia em teste: 0.3405


In [ ]:
f, ax = plt.subplot()
ax.plot([None] + model.history['acc'],'o-')
ax.plot([None] + model.history['val_acc'],'x-')
ax.legend(['Acc em treino', 'Acc em validação'],loc=0)
ax.set_title('Treino/validação acurácia por epoch')
ax.set_xlabel('Epoch')
ax.set_ylabel('Acurária')

In [ ]:
f, ax = plt.subplot()
ax.plot([None] + model.history['loss'],'o-')
ax.plot([None] + model.history['val_loss'],'x-')
ax.legend(['Perda em treino', 'Perda em validação'],loc=0)
ax.set_title('Treino/validação perda por epoch')
ax.set_xlabel('Epoch')
ax.set_ylabel('Perda')